<a href="https://colab.research.google.com/github/thegreatgupta/ObjectRecognition/blob/master/Google_Colab_Notebook_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Confirm TensorFlow can see the GPU

Simply select "GPU" in the Accelerator drop-down in Notebook Settings (either through the Edit menu or the command palette at cmd/ctrl-shift-P).

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Observe TensorFlow speedup on GPU relative to CPU

This example constructs a typical convolutional neural network layer over a
random image and manually places the resulting ops on either the CPU or the GPU
to compare execution speed.

In [4]:
import tensorflow as tf
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

with tf.device('/cpu:0'):
  random_image_cpu = tf.random_normal((100, 100, 100, 3))
  net_cpu = tf.layers.conv2d(random_image_cpu, 32, 7)
  net_cpu = tf.reduce_sum(net_cpu)

with tf.device('/gpu:0'):
  random_image_gpu = tf.random_normal((100, 100, 100, 3))
  net_gpu = tf.layers.conv2d(random_image_gpu, 32, 7)
  net_gpu = tf.reduce_sum(net_gpu)

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def cpu():
  sess.run(net_cpu)
  
def gpu():
  sess.run(net_gpu)
  
# Runs the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

sess.close()

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
10.063865595000038
GPU (s):
0.1937223529999983
GPU speedup over CPU: 51x


### Clone the Object Recognition Project from Github to fetch the Dataset and Start the training.

In [5]:
!git clone https://github.com/thegreatgupta/ObjectRecognition.git

Cloning into 'ObjectRecognition'...
remote: Enumerating objects: 6879, done.
remote: Counting objects: 100% (6879/6879), done.
remote: Compressing objects: 100% (6866/6866), done.
remote: Total 6879 (delta 36), reused 6838 (delta 12), pack-reused 0
Receiving objects: 100% (6879/6879), 123.69 MiB | 31.35 MiB/s, done.
Resolving deltas: 100% (36/36), done.
Checking out files: 100% (6951/6951), done.


In [16]:
!ls

ObjectRecognition  sample_data


In [17]:
cd ObjectRecognition

/content/ObjectRecognition


In [18]:
!ls dataset/training_set_test/

airplanes	   crab		    Faces_easy	 lamp	     scorpion
BACKGROUND_Google  crayfish	    ferry	 laptop      soccer_ball
bonsai		   dalmatian	    flamingo	 Leopards    starfish
brain		   dolphin	    grand_piano  llama	     stop_sign
buddha		   dragonfly	    hawksbill	 lotus	     sunflower
butterfly	   electric_guitar  helicopter	 menorah     trilobite
car_side	   elephant	    ibis	 minaret     umbrella
chair		   euphonium	    joshua_tree  Motorbikes  watch
chandelier	   ewer		    kangaroo	 revolver    yin_yang
cougar_face	   Faces	    ketch	 schooner


### Start Training the Model Using GPU

Import Packages to the Notebook

In [19]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
#from keras.applications.xception import Xception
#from keras.applications.resnet50 import ResNet50
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.applications.imagenet_utils import decode_predictions
from keras.layers import Dense,GlobalAveragePooling2D
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Model, load_model
from keras.optimizers import Adam

Using TensorFlow backend.


Declare Instance Variables 

In [0]:
training_set = None

Create Model Method

In [0]:
def create_model(img_shape=(299, 299, 3), n_classes=50,
                   load_pretrained=False, freeze_layers_from='base_model'):
    # Decide if load pretrained weights from imagenet
    if load_pretrained:
        weights = 'imagenet'
    else:
        weights = None

    # Get base model
    #base_model = Xception(include_top=False, weights=weights,
                       #input_tensor=None, input_shape=img_shape)
    
    #base_model = ResNet50(include_top=False, weights=weights,
                       #input_tensor=None, input_shape=img_shape)
    
    base_model = MobileNet(include_top=False, weights=weights,
                       input_tensor=None, input_shape=img_shape)

    # Add final layers
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(n_classes, activation='relu')(x)
    predictions = Dense(n_classes, activation='softmax')(x)

    # This is the model we will train
    model = Model(input=base_model.input, output=predictions)
    
    # Model Summary
    # print(model.summary())
    
    # Freeze some layers
    if freeze_layers_from is not None:
        if freeze_layers_from == 'base_model':
            print ('   Freezing base model layers')
            for layer in base_model.layers:
                layer.trainable = False
        else:
            for i, layer in enumerate(model.layers):
                print(i, layer.name)
            print ('   Freezing from layer 0 to ' + str(freeze_layers_from))
            for layer in model.layers[:freeze_layers_from]:
               layer.trainable = False
            for layer in model.layers[freeze_layers_from:]:
               layer.trainable = True

    print(model.summary())
    return model

Train the Model

In [0]:
def train_model(train_set_path, val_set_path, validation_split = 0.2,
                   batch_size = 32, class_mode = 'categorical', horizontal_flip = False,
                   vertical_flip = False, rotation_range = None, target_size = (299, 299),
                   model = None, epochs = 1, learning_rate = 0.0001, loss = 'categorical_crossentropy',
				   n_classes=50):
    
    # Load Image Data Set Using Keras
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = horizontal_flip,
                                   vertical_flip = vertical_flip,
                                   rotation_range=rotation_range,
                                   validation_split=validation_split)
    
    training_set = train_datagen.flow_from_directory(train_set_path,
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 class_mode = class_mode,
                                                 subset = 'training')
    
    validation_set = train_datagen.flow_from_directory(val_set_path,
                                                 target_size = target_size,
                                                 batch_size = batch_size,
                                                 class_mode = class_mode,
                                                 subset = 'validation')
        
    # Compile Model
    # opt_rms = keras.optimizers.rmsprop(lr=learning_rate,decay=1e-6)
    adam_optimizer = Adam(lr = learning_rate)
    model.compile(optimizer = adam_optimizer, loss = loss, metrics = ['accuracy'])
    
    model.fit_generator(training_set,
                        steps_per_epoch = training_set.samples,
                        epochs = epochs,
                        validation_data = validation_set,
                        validation_steps = validation_set.samples)
        
    model.evaluate_generator(training_set, n_classes)
    model.evaluate_generator(validation_set, n_classes)
    
    return model

Model Initialization Method to Create and Train the Model on the dataset

In [0]:
def model_initialization():
	# Declare Constants
	img_shape = (128, 128, 3)
	target_size = (128, 128)
	n_classes = 49
	epochs = 1
	train_set_path = 'dataset/training_set_test' #'101_ObjectCategories'
	val_set_path = train_set_path
	validation_split = 0.50
	batch_size = 1
	horizontal_flip = True
	rotation_range = None
	learning_rate = 0.0003
	loss = 'categorical_crossentropy'

	# Build Model
	model = create_model(img_shape = img_shape,  n_classes = n_classes, load_pretrained = True)

    #model = load_model('mobilenet_ooo3_015_10_16_model_2')
	#model.load_weights('mobilenet_ooo3_015_10_16_2.h5')

	# Train the Model
	model = train_model(train_set_path = train_set_path, val_set_path = val_set_path, validation_split = validation_split,
						   batch_size = batch_size, horizontal_flip = horizontal_flip, rotation_range = rotation_range,
						   model = model, epochs = epochs, target_size = target_size, learning_rate = learning_rate,
                           loss = loss, n_classes = n_classes)

	model.save_weights('saved_model/20190317/weight_01.h5')
	model.save('saved_model/20190317/model_01')
	json_string = model.to_json()
	f = open('saved_model/20190317/json_model_01.json', 'w+')
	f.write(json_string)
	f.close()

Call Model Initialization Method to start

In [0]:
model_initialization()